In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#importing required libraries
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [15]:
df=pd.read_csv("/content/drive/My Drive/finaldataset.csv")  # Loading the dataset to a dataframe
df.head()

,Unnamed: 0,Date,subjectivity,pos,neg,neu,compound,Open,High,Low,Close,Volume,Adj Close,change in percentage (%),Label
0,0,2008-08-08,0.15484,0.08800,0.24204,0.66984,-0.309516,11432.089844,11759.959961,11388.040039,11734.320312,212830000.0,11734.320312,2.643703,1
1,1,2008-08-11,0.16664,0.10640,0.26608,0.62756,-0.170236,11729.669922,11867.110352,11675.530273,11782.349609,183190000.0,11782.349609,0.449115,1
2,2,2008-08-12,0.34480,0.10252,0.21792,0.67952,-0.207112,11781.700195,11782.349609,11601.519531,11642.469727,173590000.0,11642.469727,-1.181752,0
3,3,2008-08-13,0.15584,0.10040,0.16168,0.73792,-0.103452,11632.809570,11633.780273,11453.339844,11532.959961,182550000.0,11532.959961,-0.858345,0
4,4,2008-08-14,0.27080,0.12120,0.22724,0.65160,-0.166048,11532.070312,11718.280273,11450.889648,11615.929688,159790000.0,11615.929688,0.727184,1


In [0]:
dates=df['Date'] #saving dates feature 

In [6]:
news=pd.read_csv("/content/drive/My Drive/RedditNews.csv") # Loading the dataset to a dataframe
news.head()

,Date,News
0,2016-07-01,A 117-year-old woman in Mexico City finally re...
1,2016-07-01,IMF chief backs Athens as permanent Olympic host
2,2016-07-01,"The president of France says if Brexit won, so..."
3,2016-07-01,British Man Who Must Give Police 24 Hours' Not...
4,2016-07-01,100+ Nobel laureates urge Greenpeace to stop o...


In [0]:
news=news.reset_index() #reseting the index

In [0]:
news = news[::-1].reset_index(drop=True) #reverseing the indexing of the dataframe
   

In [0]:
news['Date'] = pd.to_datetime(news['Date']) #changing the datatype of date feature

In [11]:
price = []
headlines = []

for row in df.iterrows():
    daily_headlines = []
    date = row[1]['Date']
    price.append(row[1]['Open'])
    for row_ in news[news.Date==date].iterrows(): #collecting all the rows having same dates into a list
        daily_headlines.append(row_[1]['News'])
    
    # Track progress
    headlines.append(daily_headlines)
    if len(price) % 500 == 0:
        print(len(price))

500
1000
1500


In [56]:
headlines[0]

'ozone layer hole seems to be healing   us  amp  uk team shows it s shrunk  amp  may slowly recover   if you had to have an ozone hole anywhere in the world  it d be antarctica because its not teeming with life  it showed us if we didnt back off with these chemicals  wed have a crisis   a hindu temple worker has been killed by three men on a motorcycle  local police have said  more than    people have been killed in attacks blamed on islamist militants in bangladesh since february       including secular bloggers  academics  gay rights activists and members of religious minorities  venezuela  where anger over food shortages is still mounting  continued to be roiled this week by angry protests and break ins of grocery stores and businesses that have left five dead  at least    injured and     arrested  according to various news reports  spain arrests three pakistanis accused of promoting militancy philippines president rodrigo duterte urges people to kill drug addicts   duterte      won

In [0]:
df2=pd.DataFrame(headlines) #converting a list to dataframe

In [13]:
#renaming the column names 
list1= [i for i in range(25)]
new_Index=[str(i) for i in list1]
df2.columns= new_Index
df2.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,Ozone layer hole seems to be healing - US &amp...,A Hindu temple worker has been killed by three...,"Venezuela, where anger over food shortages is ...",Spain arrests three Pakistanis accused of prom...,Philippines president Rodrigo Duterte urges pe...,Mass fish kill in Vietnam solved as Taiwan ste...,U.S. sailors detained by Iran spoke too much u...,India gets $1 billion loan from World Bank for...,S&amp;P revises European Union credit rating t...,"The United States has placed Myanmar, Uzbekist...",U.N. panel moves to curb bias against L.G.B.T....,"Iran celebrates American Human Rights Week, mo...",Taiwanese warship accidentally fires missile t...,Abbas PLO Faction Calls Killer of 13-Year-Old ...,France Cracks Down on Factory Farms - A viral ...,China kills millions of innocent meditators fo...,Switzerland denies Muslim girls citizenship af...,"Facebook wins privacy case, can track any Belg...",Austria's highest court annuls presidential el...,Brazil: Huge spike in number of police killing...,100+ Nobel laureates urge Greenpeace to stop o...,British Man Who Must Give Police 24 Hours' Not...,"The president of France says if Brexit won, so...",IMF chief backs Athens as permanent Olympic host,A 117-year-old woman in Mexico City finally re...
1,Law Enforcement Sources: Gun Used in Paris Ter...,UEFA says no minutes of silence for Istanbul v...,We will be swimming in ridicule - French beach...,"US permitted 1,200 offshore fracks in Gulf of ...",British pedophile sentenced to 85 years in US ...,More Than 100 Nobel Laureates Call Out Greenpe...,Calls to suspend Saudi Arabia from UN Human Ri...,The men who carried out Tuesday's terror attac...,Mounting evidence suggests 'hobbits' were wipe...,Googles free wifi at Indian railway stations i...,Groups urge U.N. to suspend Saudi Arabia from ...,CIA Gave Romania Millions to Host Secret Prisons,Japan's top court has approved blanket surveil...,Cannabis compounds could treat Alzheimer's by ...,Turkish Cop Who Took Down Istanbul Gunman Hail...,US airstrikes kill at least 250 ISIS fighters ...,Afghan suicide bomber 'kills up to 40' - BBC News,Republic of Ireland fans to be awarded medal f...,Puerto Rico will default on $1 billion of debt...,Palestinian terrorist stabs israeli teen girl ...,Switzerland denies citizenship to Muslim immig...,Six gay men in Ivory Coast were abused and for...,Boris Johnson says he will not run for Tory pa...,Stephen Hawking says pollution and 'stupidity'...,Jamaica proposes marijuana dispensers for tour...
2,Tiny New Zealand town with 'too many jobs' lau...,BBC News: Labour MPs pass Corbyn no-confidence...,Extremist violence on the rise in Germany: Dom...,Google Found Disastrous Symantec and Norton Vu...,More than 40 killed in Yemen suicide attacks,Rupert Murdoch describes Brexit as 'wonderful'...,Emaciated lions in Taiz Zoo are trapped in blo...,Car bomb and Anti-Islamic attack on Mosque in ...,The land under Beijing is sinking by as much a...,"Escape Tunnel, Dug by Hand, Is Found at Holoca...",Islamic State bombings in southern Yemen kill ...,Brexiteer Nigel Farage To EU: 'You're Not Laug...,Former Belgian Prime Minister ridicules Nigel ...,The Philippine president-elect said Monday he ...,A chatbot programmed by a British teenager has...,99-Million-Yr-Old Bird Wings Found Encased in ...,Iceland historian Johannesson wins presidentia...,UK shares and pound continue to recover,Mexican Authorities Sexually Torture Women in ...,A Muslim Shop in the UK Was Just Firebombed Wh...,British Labor Party leader Jeremy Corbyn loses...,Devastated: scientists too late to captive bre...,UK must accept freedom of movement to access E...,Yemeni former president: Terrorism is the offs...,Explosion At Airport In Istanbul
3,Mexico's security forces routinely use 'sexual...,8 Suicide Bombers Strike Lebanon,Brexit: Gibraltar in talks with Scotland to st...,"""Romanians for Remainians"" offering a new home...",Jean-Clau

In [30]:
#merging the dates to df2n
merged = pd.concat([dates, df2], axis=1, sort=False)
merged

,Date,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,2008-08-08,Ozone layer hole seems to be healing - US &amp...,A Hindu temple worker has been killed by three...,"Venezuela, where anger over food shortages is ...",Spain arrests three Pakistanis accused of prom...,Philippines president Rodrigo Duterte urges pe...,Mass fish kill in Vietnam solved as Taiwan ste...,U.S. sailors detained by Iran spoke too much u...,India gets $1 billion loan from World Bank for...,S&amp;P revises European Union credit rating t...,"The United States has placed Myanmar, Uzbekist...",U.N. panel moves to curb bias against L.G.B.T....,"Iran celebrates American Human Rights Week, mo...",Taiwanese warship accidentally fires missile t...,Abbas PLO Faction Calls Killer of 13-Year-Old ...,France Cracks Down on Factory Farms - A viral ...,China kills millions of innocent meditators fo...,Switzerland denies Muslim girls citizenship af...,"Facebook wins privacy case, can track any Belg...",Austria's highest court annuls presidential el...,Brazil: Huge spike in number of police killing...,100+ Nobel laureates urge Greenpeace to stop o...,British Man Who Must Give Police 24 Hours' Not...,"The president of France says if Brexit won, so...",IMF chief backs Athens as permanent Olympic host,A 117-year-old woman in Mexico City finally re...
1,2008-08-11,Law Enforcement Sources: Gun Used in Paris Ter...,UEFA says no minutes of silence for Istanbul v...,We will be swimming in ridicule - French beach...,"US permitted 1,200 offshore fracks in Gulf of ...",British pedophile sentenced to 85 years in US ...,More Than 100 Nobel Laureates Call Out Greenpe...,Calls to suspend Saudi Arabia from UN Human Ri...,The men who carried out Tuesday's terror attac...,Mounting evidence suggests 'hobbits' were wipe...,Googles free wifi at Indian railway stations i...,Groups urge U.N. to suspend Saudi Arabia from ...,CIA Gave Romania Millions to Host Secret Prisons,Japan's top court has approved blanket surveil...,Cannabis compounds could treat Alzheimer's by ...,Turkish Cop Who Took Down Istanbul Gunman Hail...,US airstrikes kill at least 250 ISIS fighters ...,Afghan suicide bomber 'kills up to 40' - BBC News,Republic of Ireland fans to be awarded medal f...,Puerto Rico will default on $1 billion of debt...,Palestinian terrorist stabs israeli teen girl ...,Switzerland denies citizenship to Muslim immig...,Six gay men in Ivory Coast were abused and for...,Boris Johnson says he will not run for Tory pa...,Stephen Hawking says pollution and 'stupidity'...,Jamaica proposes marijuana dispensers for tour...
2,2008-08-12,Tiny New Zealand town with 'too many jobs' lau...,BBC News: Labour MPs pass Corbyn no-confidence...,Extremist violence on the rise in Germany: Dom...,Google Found Disastrous Symantec and Norton Vu...,More than 40 killed in Yemen suicide attacks,Rupert Murdoch describes Brexit as 'wonderful'...,Emaciated lions in Taiz Zoo are trapped in blo...,Car bomb and Anti-Islamic attack on Mosque in ...,The land under Beijing is sinking by as much a...,"Escape Tunnel, Dug by Hand, Is Found at Holoca...",Islamic State bombings in southern Yemen kill ...,Brexiteer Nigel Farage To EU: 'You're Not Laug...,Former Belgian Prime Minister ridicules Nigel ...,The Philippine president-elect said Monday he ...,A chatbot programmed by a British teenager has...,99-Million-Yr-Old Bird Wings Found Encased in ...,Iceland historian Johannesson wins presidentia...,UK shares and pound continue to recover,Mexican Authorities Sexually Torture Women in ...,A Muslim Shop in the UK Was Just Firebombed Wh...,British Labor Party leader Jeremy Corbyn loses...,Devastated: scientists too late to captive bre...,UK must accept freedom of movement to access E...,Yemeni former president: Terrorism is the offs...,Explosion At Airport In Istanbul
3,2008-08-13,Mexico's security forces routinely use 'sexual...,8 Suicide Bombers Strike Lebanon,Brexit: Gibraltar in talks with Scotland to st...,"""Romanians f

In [0]:
df3=df['Label']

In [31]:
#merging label feature to dtaframe for furthur prediction process
merged=pd.concat([merged,df3],axis=1)
merged

,Date,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,Label
0,2008-08-08,Ozone layer hole seems to be healing - US &amp...,A Hindu temple worker has been killed by three...,"Venezuela, where anger over food shortages is ...",Spain arrests three Pakistanis accused of prom...,Philippines president Rodrigo Duterte urges pe...,Mass fish kill in Vietnam solved as Taiwan ste...,U.S. sailors detained by Iran spoke too much u...,India gets $1 billion loan from World Bank for...,S&amp;P revises European Union credit rating t...,"The United States has placed Myanmar, Uzbekist...",U.N. panel moves to curb bias against L.G.B.T....,"Iran celebrates American Human Rights Week, mo...",Taiwanese warship accidentally fires missile t...,Abbas PLO Faction Calls Killer of 13-Year-Old ...,France Cracks Down on Factory Farms - A viral ...,China kills millions of innocent meditators fo...,Switzerland denies Muslim girls citizenship af...,"Facebook wins privacy case, can track any Belg...",Austria's highest court annuls presidential el...,Brazil: Huge spike in number of police killing...,100+ Nobel laureates urge Greenpeace to stop o...,British Man Who Must Give Police 24 Hours' Not...,"The president of France says if Brexit won, so...",IMF chief backs Athens as permanent Olympic host,A 117-year-old woman in Mexico City finally re...,1
1,2008-08-11,Law Enforcement Sources: Gun Used in Paris Ter...,UEFA says no minutes of silence for Istanbul v...,We will be swimming in ridicule - French beach...,"US permitted 1,200 offshore fracks in Gulf of ...",British pedophile sentenced to 85 years in US ...,More Than 100 Nobel Laureates Call Out Greenpe...,Calls to suspend Saudi Arabia from UN Human Ri...,The men who carried out Tuesday's terror attac...,Mounting evidence suggests 'hobbits' were wipe...,Googles free wifi at Indian railway stations i...,Groups urge U.N. to suspend Saudi Arabia from ...,CIA Gave Romania Millions to Host Secret Prisons,Japan's top court has approved blanket surveil...,Cannabis compounds could treat Alzheimer's by ...,Turkish Cop Who Took Down Istanbul Gunman Hail...,US airstrikes kill at least 250 ISIS fighters ...,Afghan suicide bomber 'kills up to 40' - BBC News,Republic of Ireland fans to be awarded medal f...,Puerto Rico will default on $1 billion of debt...,Palestinian terrorist stabs israeli teen girl ...,Switzerland denies citizenship to Muslim immig...,Six gay men in Ivory Coast were abused and for...,Boris Johnson says he will not run for Tory pa...,Stephen Hawking says pollution and 'stupidity'...,Jamaica proposes marijuana dispensers for tour...,1
2,2008-08-12,Tiny New Zealand town with 'too many jobs' lau...,BBC News: Labour MPs pass Corbyn no-confidence...,Extremist violence on the rise in Germany: Dom...,Google Found Disastrous Symantec and Norton Vu...,More than 40 killed in Yemen suicide attacks,Rupert Murdoch describes Brexit as 'wonderful'...,Emaciated lions in Taiz Zoo are trapped in blo...,Car bomb and Anti-Islamic attack on Mosque in ...,The land under Beijing is sinking by as much a...,"Escape Tunnel, Dug by Hand, Is Found at Holoca...",Islamic State bombings in southern Yemen kill ...,Brexiteer Nigel Farage To EU: 'You're Not Laug...,Former Belgian Prime Minister ridicules Nigel ...,The Philippine president-elect said Monday he ...,A chatbot programmed by a British teenager has...,99-Million-Yr-Old Bird Wings Found Encased in ...,Iceland historian Johannesson wins presidentia...,UK shares and pound continue to recover,Mexican Authorities Sexually Torture Women in ...,A Muslim Shop in the UK Was Just Firebombed Wh...,British Labor Party leader Jeremy Corbyn loses...,Devastated: scientists too late to captive bre...,UK must accept freedom of movement to access E...,Yemeni former president: Terrorism is the offs...,Explosion At Airport In Istanbul,0
3,2008-08-13,Mexico's security forces routinely use 'sexual...,8 Suicide Bombers Strike Lebanon,Brexit: Gibraltar in talks with Scotland to st...,""

In [0]:
#splitting of data into train and test
train = merged[merged['Date'] < '20150101']
test = merged[merged['Date'] > '20141231']

In [34]:
#removing special characters
slicedData= train.iloc[:,1:26]
slicedData.replace(to_replace="[^a-zA-Z]", value=" ", regex=True, inplace=True)

# Convertng headlines to lower case
for index in new_Index:
    slicedData[index]=slicedData[index].astype(str).str.lower()
slicedData.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,ozone layer hole seems to be healing us amp...,a hindu temple worker has been killed by three...,venezuela where anger over food shortages is ...,spain arrests three pakistanis accused of prom...,philippines president rodrigo duterte urges pe...,mass fish kill in vietnam solved as taiwan ste...,u s sailors detained by iran spoke too much u...,india gets billion loan from world bank for...,s amp p revises european union credit rating t...,the united states has placed myanmar uzbekist...,u n panel moves to curb bias against l g b t ...,iran celebrates american human rights week mo...,taiwanese warship accidentally fires missile t...,abbas plo faction calls killer of year old ...,france cracks down on factory farms a viral ...,china kills millions of innocent meditators fo...,switzerland denies muslim girls citizenship af...,facebook wins privacy case can track any belg...,austria s highest court annuls presidential el...,brazil huge spike in number of police killing...,nobel laureates urge greenpeace to stop o...,british man who must give police hours not...,the president of france says if brexit won so...,imf chief backs athens as permanent olympic host,a year old woman in mexico city finally re...


In [0]:
headlines = []
for row in range(0,len(slicedData.index)):
    headlines.append(' '.join(str(x) for x in slicedData.iloc[row,0:25]))

Logistic Regresson model using count vectorization and parameter n gram

In [0]:
#ngram (1,1)

In [57]:
basicvectorizer = CountVectorizer(ngram_range=(1,1))
basictrain = basicvectorizer.fit_transform(headlines)
basicmodel = LogisticRegression()
basicmodel = basicmodel.fit(basictrain, train["Label"]) 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,1:26]))
basictest = basicvectorizer.transform(testheadlines)
predictions = basicmodel.predict(basictest)

In [59]:
print(basictrain.shape)

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix

print (classification_report(test["Label"], predictions))
print (accuracy_score(test["Label"], predictions))

(1863, 32475)
              precision    recall  f1-score   support

           0       0.82      0.85      0.84       183
           1       0.86      0.83      0.84       195

    accuracy                           0.84       378
   macro avg       0.84      0.84      0.84       378
weighted avg       0.84      0.84      0.84       378

0.8386243386243386


In [0]:
#ngram (2,2)

In [55]:
basictest2 = basicvectorizer2.transform(testheadlines)
predictions2 = basicmodel2.predict(basictest2)
basicmodel2 = LogisticRegression()
basicmodel2 = basicmodel2.fit(basictrain2, train["Label"])
basicmodel2 = LogisticRegression()
basicmodel2 = basicmodel2.fit(basictrain2, train["Label"])
print(basictrain2.shape)
print (classification_report(test["Label"], predictions2))
print (accuracy_score(test["Label"], predictions2))

(1863, 409523)
              precision    recall  f1-score   support

           0       0.91      0.80      0.85       183
           1       0.83      0.93      0.88       195

    accuracy                           0.87       378
   macro avg       0.87      0.87      0.87       378
weighted avg       0.87      0.87      0.87       378

0.8677248677248677


In [0]:
#ngram (3,3)

In [54]:
basicvectorizer3 = CountVectorizer(ngram_range=(3,3))
basictrain3 = basicvectorizer3.fit_transform(headlines)
print(basictrain3.shape)

basicmodel3 = LogisticRegression()
basicmodel3 = basicmodel3.fit(basictrain3, train["Label"])

basictest3 = basicvectorizer3.transform(testheadlines)
predictions3 = basicmodel3.predict(basictest3)

print (classification_report(test["Label"], predictions3))
print (accuracy_score(test["Label"], predictions3))

(1863, 702474)
              precision    recall  f1-score   support

           0       0.99      0.73      0.84       183
           1       0.79      0.99      0.88       195

    accuracy                           0.86       378
   macro avg       0.89      0.86      0.86       378
weighted avg       0.89      0.86      0.86       378

0.8624338624338624
